In [77]:
import pandas as pd

## 1. Top 10 batsmen based on past 3 years total runs scored.

In [78]:

batsmen=pd.read_csv("./Ipl Datasets/fact_bating_summary.csv")
batsmen

,match_id,match,teamInnings,battingPos,batsmanName,out/not_out,runs,balls,4s,6s,SR
0,T203817,Super Kings Vs KKR,Super Kings,1,RuturajGaikwad,out,32,27,3,1,118.51
1,T203817,Super Kings Vs KKR,Super Kings,2,FafduPlessis,out,86,59,7,3,145.76
2,T203817,Super Kings Vs KKR,Super Kings,3,RobinUthappa,out,31,15,0,3,206.66
3,T203817,Super Kings Vs KKR,Super Kings,4,MoeenAli,not_out,37,20,2,3,185.00
4,T203817,Super Kings Vs KKR,KKR,1,ShubmanGill,out,51,43,6,0,118.60
...,...,...,...,...,...,...,...,...,...,...,...
3263,T201566,Super Kings Vs Titans,Titans,3,SaiSudharsan,out,22,17,3,0,129.41
3264,T201566,Super Kings Vs Titans,Titans,4,HardikPandya,out,8,11,0,0,72.72
3265,T201566,Super Kings Vs Titans,Titans,5,VijayShankar,out,27,21,2,1,128.57
3266,T201566,Super Kings Vs Titans,Titans,6,RahulTewatia,not_out,15,14,1,1,107.14


In [79]:
top_batsmens = (batsmen.groupby('batsmanName')["runs"].sum()
                .reset_index()
                .sort_values(by='runs', ascending=False))
                
top_batsmens.head(10)

,batsmanName,runs
222,ShubmanGill,1851
66,FafduPlessis,1831
197,RuturajGaikwad,1593
97,KLRahul,1516
95,JosButtler,1509
216,ShikharDhawan,1392
251,ViratKohli,1385
204,SanjuSamson,1304
230,SuryakumarYadav,1225
69,GlennMaxwell,1214


## 2. Top 10 batsmen based on past 3 years batting average. (min 60 balls faced in each season)

In [80]:

match_sum=pd.read_csv("./Ipl Datasets/dim_match_summary.csv")
match_sum

,team1,team2,winner,margin,matchDate,match_id
0,Super Kings,KKR,Super Kings,27 runs,"Oct 15, 2021",T203817
1,Capitals,KKR,KKR,3 wickets,"Oct 13, 2021",T206442
2,RCB,KKR,KKR,4 wickets,"Oct 11, 2021",T208597
3,Capitals,Super Kings,Super Kings,4 wickets,"Oct 10, 2021",T201819
4,Capitals,RCB,RCB,7 wickets,"Oct 8, 2021",T205942
...,...,...,...,...,...,...
201,Mumbai,RCB,RCB,8 wickets,"Apr 2, 2023",T202485
202,Royals,Sunrisers,Royals,72 runs,"Apr 2, 2023",T209117
203,Super Giants,Capitals,Super Giants,50 runs,"Apr 1, 2023",T208512
204,Punjab Kings,KKR,Punjab Kings,7 runs,"Apr 1, 2023",T202853


In [81]:
import pandas as pd

# Load datasets
batting = pd.read_csv("./Ipl Datasets/fact_bating_summary.csv")
matches = pd.read_csv("./Ipl Datasets/dim_match_summary.csv")

# Merge matchDate into batting dataset
batting = batting.merge(matches[["match_id", "matchDate"]], on="match_id", how="left")

batting['year'] = pd.to_datetime(batting['matchDate'], errors='coerce').dt.year

# Aggregate per batsman per year
stats = (batting.groupby(['year', 'batsmanName'])
                 .agg(
                     runs=('runs', 'sum'),
                     balls=('balls', 'sum'),
                     outs=('out/not_out', lambda x: (x=='out').sum())
                 ))

# Filter by minimum 60 balls per season
stats = stats[stats['balls'] >= 60]

# Compute batting average per year
stats['batting_avg'] = stats['runs'] / stats['outs']

print(stats)
valid_batsmen = stats.reset_index().groupby('batsmanName')['year'].nunique()
valid_batsmen = valid_batsmen[valid_batsmen == 3].index

stats = stats.loc[stats.index.get_level_values('batsmanName').isin(valid_batsmen)]

# Step 3: compute overall batting average = total runs / total outs across 3 years
overall = stats.groupby('batsmanName').agg(
    total_runs=('runs','sum'),
    total_outs=('outs','sum'),
    total_balls=('balls','sum')
)

overall['batting_avg'] = overall['total_runs'] / overall['total_outs']

# Top 10 batsmen by overall batting average
top_10 = overall.sort_values('batting_avg', ascending=False).head(10)

print(top_10)

                         runs  balls  outs  batting_avg
year   batsmanName                                     
2021.0 ABdeVilliers       313    211    10    31.300000
       AbdulSamad         111     87     9    12.333333
       AbhishekSharma      93     69     5    18.600000
       AidenMarkram       146    119     5    29.200000
       AmbatiRayudu       257    170    10    25.700000
...                       ...    ...   ...          ...
2023.0 VijayShankar       301    188     8    37.625000
       ViratKohli         639    457    12    53.250000
       WashingtonSundar    60     60     4    15.000000
       WriddhimanSaha     317    248    15    21.133333
       YashasviJaiswal    625    382    13    48.076923

[218 rows x 4 columns]
                 total_runs  total_outs  total_balls  batting_avg
batsmanName                                                      
KLRahul                1516          30         1148    50.533333
FafduPlessis           1831          42         13

## 3. Top 10 batsmen based on past 3 years strike rate (min 60 balls faced in each season)

In [82]:
valid_batsmen = stats.reset_index().groupby('batsmanName')['year'].nunique()
valid_batsmen = valid_batsmen[valid_batsmen == 3].index

stats = stats.loc[stats.index.get_level_values('batsmanName').isin(valid_batsmen)]

# Step 3: compute overall batting average = total runs / total outs across 3 years
overall = stats.groupby('batsmanName').agg(
    total_runs=('runs','sum'),
    total_outs=('outs','sum'),
    total_balls=('balls','sum')
)

overall['strike_rate'] = overall['total_runs'] / overall['total_balls']*100

# Top 10 batsmen by overall batting average
Strikerate = overall.sort_values('strike_rate', ascending=False).head(10)
Strikerate

,total_runs,total_outs,total_balls,strike_rate
batsmanName,,,,
GlennMaxwell,1214,35,752,161.436170
SuryakumarYadav,1225,35,763,160.550459
AndreRussell,745,27,468,159.188034
ShimronHetmyer,854,21,543,157.274401
NicholasPooran,729,30,464,157.112069
PrithviShaw,815,32,532,153.195489
DineshKarthik,693,28,454,152.643172
YashasviJaiswal,1132,33,744,152.150538
JosButtler,1509,36,1027,146.932814


## 4. Top 10 bowlers based on past 3 years total wickets taken.

In [83]:
bowler=pd.read_csv("./Ipl Datasets/fact_bowling_summary.csv")

top_wickets = (bowler.groupby('bowlerName')["wickets"].sum()
                .reset_index()
                .sort_values(by='wickets',ascending=False))

top_wickets.head(10)

,bowlerName,wickets
110,MohammedShami,67
201,YuzvendraChahal,66
54,HarshalPatel,65
141,RashidKhan,63
20,AveshKhan,47
19,ArshdeepSingh,45
75,KagisoRabada,45
189,VarunChakravarthy,44
163,ShardulThakur,43
181,TrentBoult,42


## 5. Top 10 bowlers based on past 3 years bowling average. (min 60 balls bowled in each season)

In [84]:
# Load datasets
bowler=pd.read_csv("./Ipl Datasets/fact_bowling_summary.csv")
matches = pd.read_csv("./Ipl Datasets/dim_match_summary.csv")

# Merge matchDate into bowler dataset
bowler = bowler.merge(matches[["match_id", "matchDate"]], on="match_id", how="left")

bowler['year'] = pd.to_datetime(bowler['matchDate'], errors='coerce').dt.year

# Aggregate per bowler per year
stats = (bowler.groupby(['year', 'bowlerName'])
                 .agg(
                     runs=('runs', 'sum'),
                     wickets=('wickets', 'sum'),
                     overs=('overs', 'sum'),
                    ))

# Filter by minimum 60 balls per season
stats = stats[stats['overs'] >= 10]

# Compute bowling average per year
stats['bowling_avg'] = stats['runs'] / stats['wickets']

print(stats)

valid_bowling = stats.reset_index().groupby('bowlerName')['year'].nunique()
valid_bowling = valid_bowling[valid_bowling == 3].index

stats = stats.loc[stats.index.get_level_values('bowlerName').isin(valid_bowling)]

# Step 3: compute overall bowling average = total runs / total wickets across 3 years
overall = stats.groupby('bowlerName').agg(
    total_runs=('runs','sum'),
    total_overs=('overs','sum'),
    total_wickets=('wickets','sum')
)

overall['bowling_avg'] = overall['total_runs'] / overall['total_wickets']

# Top 10 bowler by overall bowling average
bowler_10 = overall.sort_values('bowling_avg').head(10)

bowler_10

                         runs  wickets  overs  bowling_avg
year   bowlerName                                         
2021.0 AdamMilne          131        3   14.0    43.666667
       AmitMishra          78        5   10.0    15.600000
       AndreRussell       188       11   19.0    17.090909
       AnrichNortje       187       12   30.2    15.583333
       ArshdeepSingh      342       18   40.8    19.000000
...                       ...      ...    ...          ...
2023.0 WashingtonSundar   146        3   17.4    48.666667
       WayneParnell       236        9   25.1    26.222222
       YashDayal          165        2   14.0    82.500000
       YashThakur         289       13   31.5    22.230769
       YuzvendraChahal    432       21   52.5    20.571429

[233 rows x 4 columns]


,total_runs,total_overs,total_wickets,bowling_avg
bowlerName,,,,
AndreRussell,638,61.8,35,18.228571
YuzvendraChahal,1333,173.5,66,20.196970
RashidKhan,1273,179.5,63,20.206349
HarshalPatel,1323,155.7,65,20.353846
MohammedShami,1376,175.4,67,20.537313
AveshKhan,1115,133.4,47,23.723404
KagisoRabada,1069,124.0,45,23.755556
MoeenAli,501,70.7,21,23.857143
AnrichNortje,768,92.4,31,24.774194


## 6. Top 10 bowlers based on past 3 years economy rate. (min 60 balls bowled in each season)

In [95]:
# Load datasets
bowler=pd.read_csv("./Ipl Datasets/fact_bowling_summary.csv")
matches = pd.read_csv("./Ipl Datasets/dim_match_summary.csv")

# Merge matchDate into bowler dataset
bowler = bowler.merge(matches[["match_id", "matchDate"]], on="match_id", how="left")

bowler['year'] = pd.to_datetime(bowler['matchDate'], errors='coerce').dt.year

bowler['full_overs'] = bowler['overs'].astype(int)                  # integer part
bowler['balls_part'] = ((bowler['overs'] - bowler['full_overs']) * 10).round().astype(int)

# Total balls = overs*6 + balls
bowler['balls'] = bowler['full_overs'] * 6 + bowler['balls_part']

# Aggregate per bowler per year
stats = (bowler.groupby(['year', 'bowlerName'])
                 .agg(
                     runs=('runs', 'sum'),
                     wickets=('wickets', 'sum'),
                     overs=('overs', 'sum'),
                     balls=("balls","sum"),
                    ))

# Filter by minimum 60 balls per season
stats = stats[stats['overs'] >= 10]
# Compute bowling_economy per year
stats['bowling_economy'] = stats['runs'] / stats['balls']

print(stats)

valid_bowling = stats.reset_index().groupby('bowlerName')['year'].nunique()
valid_bowling = valid_bowling[valid_bowling == 3].index

stats = stats.loc[stats.index.get_level_values('bowlerName').isin(valid_bowling)]

# Step 3: compute overall bowling_economy = total runs / total overs across 3 years
overall = stats.groupby('bowlerName').agg(
    total_runs=('runs','sum'),
    total_balls=('balls','sum'),
    total_wickets=('wickets','sum')
)

overall['bowling_economy'] = overall['total_runs'] / overall['total_balls']

# Top 10 bowler by overall bowling_economy
bowler_10 = overall.sort_values('bowling_economy').head(10)

bowler_10

                         runs  wickets  overs  balls  bowling_economy
year   bowlerName                                                    
2021.0 AdamMilne          131        3   14.0     84         1.559524
       AmitMishra          78        5   10.0     60         1.300000
       AndreRussell       188       11   19.0    114         1.649123
       AnrichNortje       187       12   30.2    182         1.027473
       ArshdeepSingh      342       18   40.8    248         1.379032
...                       ...      ...    ...    ...              ...
2023.0 WashingtonSundar   146        3   17.4    106         1.377358
       WayneParnell       236        9   25.1    151         1.562914
       YashDayal          165        2   14.0     84         1.964286
       YashThakur         289       13   31.5    191         1.513089
       YuzvendraChahal    432       21   52.5    317         1.362776

[233 rows x 5 columns]


,total_runs,total_balls,total_wickets,bowling_economy
bowlerName,,,,
SunilNarine,1056,960,36,1.100000
MoeenAli,501,427,21,1.173302
RashidKhan,1273,1079,63,1.179796
AxarPatel,939,792,30,1.185606
RavindraJadeja,976,792,36,1.232323
KrunalPandya,843,679,24,1.241532
RavichandranAshwin,1175,940,33,1.250000
VarunChakravarthy,1209,958,44,1.262004
HarpreetBrar,503,397,18,1.267003


## 7. Top 5 batsmen based on past 3 years boundary % (fours and sixes).

In [86]:
import pandas as pd

# Load datasets
batting = pd.read_csv("./Ipl Datasets/fact_bating_summary.csv")
matches = pd.read_csv("./Ipl Datasets/dim_match_summary.csv")

# Merge matchDate into batting dataset
batting = batting.merge(matches[["match_id", "matchDate"]], on="match_id", how="left")

batting['year'] = pd.to_datetime(batting['matchDate'], errors='coerce').dt.year

# Aggregate per batsman per year
stats = (batting.groupby(['year', 'batsmanName'])
                 .agg(
                     runs=('runs', 'sum'),
                     balls=('balls', 'sum'),
                    fours=('4s','sum'),
                    six=("6s",'sum'))
                 )

# Filter by minimum 60 balls per season
stats = stats[stats['balls'] >= 60]

# Compute batting average per year
stats['boundary'] = stats['fours'] + stats['six']

print(stats)
valid_batsmen = stats.reset_index().groupby('batsmanName')['year'].nunique()
valid_batsmen = valid_batsmen[valid_batsmen == 3].index

stats = stats.loc[stats.index.get_level_values('batsmanName').isin(valid_batsmen)]

# Step 3: compute overall batting average = total runs / total outs across 3 years
overall = stats.groupby('batsmanName').agg(
    total_runs=('runs','sum'),
    total_fours=('fours','sum'),
    total_six=("six",'sum'),
    total_balls=('balls','sum'),
)

overall['boundary%'] = (overall['total_fours']*4 + overall['total_six']*6) / overall['total_runs'] * 100

# Top 10 batsmen by overall boundary%'
top_5 = overall.sort_values('boundary%', ascending=False).head(5)

top_5

                         runs  balls  fours  six  boundary
year   batsmanName                                        
2021.0 ABdeVilliers       313    211     23   16        39
       AbdulSamad         111     87      4    8        12
       AbhishekSharma      93     69      7    4        11
       AidenMarkram       146    119     12    4        16
       AmbatiRayudu       257    170     16   17        33
...                       ...    ...    ...  ...       ...
2023.0 VijayShankar       301    188     27   15        42
       ViratKohli         639    457     65   16        81
       WashingtonSundar    60     60      6    0         6
       WriddhimanSaha     317    248     42    7        49
       YashasviJaiswal    625    382     82   26       108

[218 rows x 5 columns]


,total_runs,total_fours,total_six,total_balls,boundary%
batsmanName,,,,,
AndreRussell,745,45,64,468,75.704698
YashasviJaiswal,1132,142,46,744,74.558304
PrithviShaw,815,102,28,532,70.674847
JosButtler,1509,152,72,1027,68.919814
GlennMaxwell,1214,108,67,752,68.698517


## 8. Top 5 bowlers based on past 3 years dot ball %.

In [100]:
# Load datasets
bowler=pd.read_csv("./Ipl Datasets/fact_bowling_summary.csv")
matches = pd.read_csv("./Ipl Datasets/dim_match_summary.csv")

# Merge matchDate into bowler dataset
bowler = bowler.merge(matches[["match_id", "matchDate"]], on="match_id", how="left")

bowler['year'] = pd.to_datetime(bowler['matchDate'], errors='coerce').dt.year

bowler['full_overs'] = bowler['overs'].astype(int)                  # integer part
bowler['balls_part'] = ((bowler['overs'] - bowler['full_overs']) * 10).round().astype(int)

# Total balls = overs*6 + balls
bowler['balls'] = bowler['full_overs'] * 6 + bowler['balls_part']

# Aggregate per bowler per year
stats = (bowler.groupby(['year', 'bowlerName'])
                 .agg(
                     dots=('0s', 'sum'),
                     balls=("balls","sum"),
                    ))

# Filter by minimum 60 balls per season
stats = stats[stats['balls'] >= 60]
# Compute bowling_economy per year
stats['bowling_economy'] = stats['dots'] / stats['balls']

print(stats)

valid_bowling = stats.reset_index().groupby('bowlerName')['year'].nunique()
valid_bowling = valid_bowling[valid_bowling == 3].index

stats = stats.loc[stats.index.get_level_values('bowlerName').isin(valid_bowling)]

# Step 3: compute overall bowling_economy = total runs / total overs across 3 years
overall = stats.groupby('bowlerName').agg(
    total_dots=('dots','sum'),
    total_balls=('balls','sum'),
)

overall['dot_bowls'] = overall['total_dots'] / overall['total_balls'] * 100

# Top 10 bowler by overall bowling_economy
dotballs_5 = overall.sort_values('dot_bowls').head(5)

dotballs_5

                         dots  balls  bowling_economy
year   bowlerName                                    
2021.0 AdamMilne           31     84         0.369048
       AmitMishra          17     60         0.283333
       AndreRussell        39    114         0.342105
       AnrichNortje        84    182         0.461538
       ArshdeepSingh       88    248         0.354839
...                       ...    ...              ...
2023.0 WashingtonSundar    35    106         0.330189
       WayneParnell        56    151         0.370861
       YashDayal           24     84         0.285714
       YashThakur          70    191         0.366492
       YuzvendraChahal     97    317         0.305994

[234 rows x 3 columns]


,total_dots,total_balls,dot_bowls
bowlerName,,,
LalitYadav,83,270,30.740741
WashingtonSundar,114,370,30.810811
RavichandranAshwin,295,940,31.382979
JasonHolder,198,611,32.405892
ChrisJordan,94,281,33.451957


## 9. Top 4 teams based on past 3 years winning %.

## 10.Top 2 teams with the highest number of wins achieved by chasing targets over the past 3 years.